In [35]:
# fastText is a library for efficient learning of word representations and sentence classification.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import fasttext
import bz2
import csv
from sklearn.metrics import roc_auc_score
import os


In [37]:
# Load the training data 
dt = bz2.BZ2File("train.ft.txt.bz2")
data = dt.readlines()
train = [x.decode('utf-8') for x in data]
print(len(train)) 

3600000


In [39]:
train[0:2]

['__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^\n',
 "__label__2 The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny.\n"]

### Data modelling

In [4]:
# Data Prep
train = pd.DataFrame(train)
train.to_csv("train.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")


In [32]:
model = fasttext.train_supervised('train.txt')

In [6]:
#print(model.words)
print(model.labels)

['__label__2', '__label__1']


### prediction

In [7]:
# Load the test data 
test = bz2.BZ2File("test.ft.txt.bz2")
test = test.readlines()
test = [x.decode('utf-8') for x in test]
print(len(test), 'number of records in the test set') 

400000 number of records in the test set


In [8]:
# To run the predict function, we need to remove the __label__1 and __label__2 from the testset.  
new = [w.replace('__label__2 ', '') for w in test]
new = [w.replace('__label__1 ', '') for w in new]

In [40]:
new = [w.replace('\n', '') for w in new]


In [10]:
# Use the predict function 
pred = model.predict(new)

In [12]:
# check the first record outputs
print(pred[0][0], 'is the predicted label')
print(pred[1][0], 'is the probability score')

['__label__2'] is the predicted label
[0.9946661] is the probability score


### Evaluation

In [13]:
# Lets recode the actual targets to 1's and 0's from both the test set and the actual predictions  
labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test]
pred_labels = [0 if x == ['__label__1'] else 1 for x in pred[0]]

# run the accuracy measure. 
print(roc_auc_score(labels, pred_labels))

0.9161699999999999


### Compress model files with quantization

When you want to save a supervised model file, fastText can compress it in order to have a much smaller model file by sacrificing only a little bit performance.

In [15]:
# Data Prep
test = pd.DataFrame(test)
test.to_csv("test.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [16]:
# with the previously trained `model` object, call :
model.quantize(input='train.txt', retrain=True)



In [19]:
#To evaluate our model by computing the precision at 1 (P@1) and the recall on a test set, we use the test function:

def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model.test('test.txt'))

N	400000
P@1	0.915
R@1	0.915


In [31]:
# save the new model :
model.save_model("model_filename.ftz")